# Figure 6, Panels E and F

In [ ]:
# 0. Import
import os
import sys

import matplotlib.pyplot as plt
from matplotlib_venn import venn3, venn2
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns

from scipy.io import mmread
from scipy.sparse import csr_matrix

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from celloracle import motif_analysis as ma
import celloracle as co
co.__version__

In [ ]:
# visualization settings
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

## Make 4,000-edge Gephi-compatible files

Only doing this for E8.5 and E9 networks. Final hairballs made in Gephi.

In [ ]:
links_wt_e85 = co.load_hdf5(file_path="./data/celloracle/e85/WT_cardiac-subset-links.celloracle.links")
links_wt_e9 = co.load_hdf5(file_path="./data/celloracle/e9/WT_cardiac-subset-links.celloracle.links")

links_ko_e85 = co.load_hdf5(file_path="./data/celloracle/e85/KO_cardiac-subset-links.celloracle.links")
links_ko_e9 = co.load_hdf5(file_path="./data/celloracle/e9/KO_cardiac-subset-links.celloracle.links")

In [ ]:
links_dict = {}

for timepoint, wt_or_ko in [['85', 'WT'], ['85', 'KO'], ['9', 'WT'], ['9', 'KO']]:
    links = co.load_hdf5(file_path=f"./data/celloracle/e{timepoint}/{wt_or_ko}_cardiac-subset-links.celloracle.links")

    links.filter_links(p=0.001, weight="coef_abs", threshold_number=4500)
    filtered_df = links.filtered_links

    for cluster in filtered_df.keys():
        df = filtered_df[cluster]
        
        if wt_or_ko == 'KO':
            df = df.drop(df[df['source'] == 'Mef2c'].index)

        df = df.sort_values('coef_abs', ascending=False).iloc[:4000,:]

        sources = df['source'].unique()
        targets = df['target'].unique()

        # Create a zero matrix
        matrix = pd.DataFrame(np.zeros((len(sources), len(targets))), index=sources, columns=targets)

        # Populate the matrix with coef_mean values
        for index, row in df.iterrows():
            matrix.at[row['source'], row['target']] = row['coef_mean']

        matrix.to_csv(f"./data/celloracle/e{timepoint}/{wt_or_ko}_{cluster}_gephi.csv")